In [5]:
import pandas as pd 
import os
from dotenv import load_dotenv
import berserk
pd.get_option("display.max_colwidth", 200) 
from datetime import datetime
# load content of env
load_dotenv()

# load the token
lichess_token = os.getenv("my_token")

# using token 
session = berserk.TokenSession(lichess_token)
client = berserk.Client(session=session)

# get profile data from my personal account
#mydata = client.users.get_public_data("julioam")
#mydata_df = pd.json_normalize(mydata).transpose()
#print(mydata_df)

# history 
#myhistoy = client.users.get_rating_history("julioam")
#myhistoy_df = pd.json_normalize(myhistoy)
#print(myhistoy_df)

# top 10 
#top10_players = client.users.get_all_top_10()
#pd.json_normalize(top10_players)

ResponseError: HTTP 401: Unauthorized: {'error': 'No such user'}

In [ ]:
# top bullet 
top10_bullet = pd.json_normalize(top10_players, record_path="bullet").rename(columns={"perfs.bullet.rating" : "bullet_rating", "perfs.bullet.progress" : "bullet_progress"})
top10_bullet

,id,username,bullet_rating,bullet_progress,title,patron,online
0,chesswarrior31,Chesswarrior31,3281,14,NaN,NaN,NaN
1,muisback,muisback,3203,-20,GM,True,NaN
2,aaryan_varshney,aaryan_varshney,3200,35,FM,NaN,NaN
3,azzaro25,Azzaro25,3200,27,GM,NaN,NaN
4,watneg,Watneg,3188,32,GM,NaN,NaN
5,howardxue,HowardXue,3177,14,NaN,NaN,NaN
6,avalongamemaster,avalongamemaster,3174,-2,GM,NaN,NaN
7,penguingim1,penguingim1,3169,11,GM,True,NaN
8,aqua_blazing,Aqua_Blazing,3162,3,NaN,NaN,True
9,mirakel05,Mirakel05,3161,16,FM,NaN,NaN


In [ ]:
# top 10 blitz
top10_blitz = pd.json_normalize(top10_players, record_path="blitz").rename(columns={"perfs.blitz.rating" : "blitz_rating", "perfs.blitz.progress" : "blitz_progress"})
top10_blitz

,id,username,title,patron,blitz_rating,blitz_progress
0,muisback,muisback,GM,True,3057,14
1,aspiringstar,aspiringstar,GM,NaN,2977,35
2,dr-cro,Dr-CRO,GM,NaN,2976,32
3,lintchevski_daniil,Lintchevski_Daniil,GM,NaN,2954,26
4,athena-pallada,athena-pallada,GM,NaN,2954,2
5,i_am_the_best1,I_am_the_best1,NaN,NaN,2943,95
6,ls_kokomo,LS_kokomo,NaN,NaN,2913,32
7,lucifermorning01,lucifermorning01,NaN,NaN,2907,-2
8,howardxue,HowardXue,NaN,NaN,2905,-2
9,evgeny_levin,Evgeny_Levin,GM,NaN,2891,25


In [ ]:
# top 10 rapid
top10_rapid = pd.json_normalize(top10_players, record_path="rapid").rename(columns={"perfs.rapid.rating" : "rapid_rating", "perfs.rapid.progress" : "rapid_progress"})
top10_rapid

,id,username,title,rapid_rating,rapid_progress,patron,online
0,drvitman,DrVitman,GM,2938,-6,NaN,NaN
1,unicorn7love,Unicorn7Love,IM,2888,7,True,NaN
2,drawdenied_twitch,DrawDenied_Twitch,GM,2868,21,NaN,NaN
3,rakhmanov_aleksandr,Rakhmanov_Aleksandr,GM,2811,30,True,NaN
4,kurald_galain,Kurald_Galain,NaN,2806,24,NaN,True
5,roman_yanchenko,Roman_Yanchenko,FM,2801,28,NaN,True
6,forester_19,Forester_19,NaN,2777,-26,NaN,NaN
7,tuzakli_egitim,Tuzakli_Egitim,FM,2767,-55,NaN,NaN
8,savvavetokhin2009,SavvaVetokhin2009,IM,2751,-34,True,True
9,konnov_oleg,Konnov_Oleg,NaN,2733,25,NaN,NaN


In [ ]:
# top 10 classic 
top10_classic = pd.json_normalize(top10_players, record_path="classical").rename(columns={"perfs.classical.rating" : "classic_rating", "perfs.classical.progress" : "classic_progress"})
top10_classic

,id,username,title,patron,classic_rating,classic_progress,online
0,unicorn7love,Unicorn7Love,IM,True,2755,6,NaN
1,kanake04,Kanake04,NaN,NaN,2465,-10,NaN
2,vlad_lazarev79,Vlad_Lazarev79,NaN,NaN,2461,12,NaN
3,ojaijoao,OjaiJoao,FM,NaN,2460,4,NaN
4,igormezentsev,igormezentsev,FM,NaN,2456,-15,NaN
5,tinea,Tinea,NaN,NaN,2454,-9,NaN
6,konnov_oleg,Konnov_Oleg,NaN,NaN,2447,0,NaN
7,laguna_tsubo,Laguna_Tsubo,NaN,True,2442,30,NaN
8,chesstheory64,ChessTheory64,FM,True,2439,71,NaN
9,teodorvasilov,TeodorVasilov,NaN,NaN,2430,56,True


In [ ]:
top10_ultrabullet = pd.json_normalize(top10_players, record_path="ultraBullet")
top10_ultrabullet 

,id,username,title,perfs.ultraBullet.rating,perfs.ultraBullet.progress,online,patron
0,aaryan_varshney,aaryan_varshney,FM,2660,26,NaN,NaN
1,blazinq,Blazinq,FM,2554,29,True,NaN
2,ohanyaneminchess,OhanyanEminChess,GM,2539,-8,NaN,NaN
3,tamojerry,tamojerry,FM,2529,15,NaN,NaN
4,ediz_gurel,Ediz_Gurel,GM,2526,-25,NaN,NaN
5,anonymousblitz,anonymousblitz,NaN,2522,25,NaN,NaN
6,mishka_the_great,Mishka_The_Great,FM,2515,23,True,NaN
7,dir07,dIr07,NaN,2511,1,NaN,True
8,olegpro1,OlegPro1,NaN,2501,30,True,True
9,mereseberserknhi,MereseBerserkNhi,NaN,2466,-18,True,NaN


In [ ]:
# top 10 bullet detail data
top10_bullet_df = pd.DataFrame()
for i in top10_bullet["id"]:
    top10_bullet_df = pd.concat([top10_bullet_df, pd.json_normalize(client.users.get_public_data(i))], ignore_index=True)

top10_bullet_df.head()

,id,username,createdAt,seenAt,url,followable,following,blocking,followsYou,perfs.ultraBullet.games,...,perfs.atomic.prog,perfs.atomic.prov,perfs.racingKings.games,perfs.racingKings.rating,perfs.racingKings.rd,perfs.racingKings.prog,perfs.racingKings.prov,profile.uscfRating,streamer.youTube.channel,playing
0,chesswarrior31,Chesswarrior31,2022-05-10 02:44:21.027000+00:00,2024-06-17 04:40:14.629000+00:00,https://lichess.org/@/Chesswarrior31,True,False,False,False,5854.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,muisback,muisback,2020-01-28 19:10:32.291000+00:00,2024-06-17 07:38:21.407000+00:00,https://lichess.org/@/muisback,True,False,False,False,1747.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aaryan_varshney,aaryan_varshney,2019-04-21 17:09:04.962000+00:00,2024-06-17 07:39:01.059000+00:00,https://lichess.org/@/aaryan_varshney,True,False,False,False,5783.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,azzaro25,Azzaro25,2016-09-20 13:30:27.051000+00:00,2024-06-14 15:02:17.927000+00:00,https://lichess.org/@/Azzaro25,True,False,False,False,276.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,watneg,Watneg,2018-12-24 20:45:36.272000+00:00,2024-06-17 07:08:32.471000+00:00,https://lichess.org/@/Watneg,True,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
top10_bullet_df.columns

Index(['id', 'username', 'createdAt', 'seenAt', 'url', 'followable',
       'following', 'blocking', 'followsYou', 'perfs.ultraBullet.games',
       ...
       'perfs.atomic.prog', 'perfs.atomic.prov', 'perfs.racingKings.games',
       'perfs.racingKings.rating', 'perfs.racingKings.rd',
       'perfs.racingKings.prog', 'perfs.racingKings.prov',
       'profile.uscfRating', 'streamer.youTube.channel', 'playing'],
      dtype='object', length=119)

In [ ]:
# top 10 blitz detail data
top10_blitz_df = pd.DataFrame()
for i in top10_blitz["id"]:
    top10_blitz_df = pd.concat([top10_blitz_df, pd.json_normalize(client.users.get_public_data(i))], ignore_index=True)

top10_blitz_df.head()

,id,username,title,patron,createdAt,seenAt,url,followable,following,blocking,...,perfs.horde.games,perfs.horde.rating,perfs.horde.rd,perfs.horde.prog,perfs.horde.prov,perfs.racingKings.games,perfs.racingKings.rating,perfs.racingKings.rd,perfs.racingKings.prog,perfs.racingKings.prov
0,muisback,muisback,GM,True,2020-01-28 19:10:32.291000+00:00,2024-06-17 07:38:21.407000+00:00,https://lichess.org/@/muisback,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aspiringstar,aspiringstar,GM,NaN,2022-06-18 04:11:21.387000+00:00,2024-06-17 05:54:23.392000+00:00,https://lichess.org/@/aspiringstar,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dr-cro,Dr-CRO,GM,NaN,2019-06-22 14:14:03.867000+00:00,2024-06-16 20:49:18.941000+00:00,https://lichess.org/@/Dr-CRO,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lintchevski_daniil,Lintchevski_Daniil,GM,NaN,2023-12-08 11:14:13.170000+00:00,2024-06-16 21:02:30.484000+00:00,https://lichess.org/@/Lintchevski_Daniil,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,athena-pallada,athena-pallada,GM,NaN,2020-03-20 09:26:30.527000+00:00,2024-06-17 07:38:52.883000+00:00,https://lichess.org/@/athena-pallada,True,False,False,...,17.0,1890.0,275.0,-55.0,True,213.0,1930.0,260.0,14.0,True


In [ ]:
# top 10 rapid detail data
top10_rapid_df = pd.DataFrame()
for i in top10_rapid["id"]:
    top10_rapid_df = pd.concat([top10_rapid_df, pd.json_normalize(client.users.get_public_data(i))], ignore_index=True)

top10_rapid_df.head()

,id,username,title,flair,createdAt,seenAt,url,followable,following,blocking,...,profile.bio,profile.lastName,perfs.ultraBullet.prov,streamer.twitch.channel,perfs.racingKings.games,perfs.racingKings.rating,perfs.racingKings.rd,perfs.racingKings.prog,perfs.racingKings.prov,playing
0,drvitman,DrVitman,GM,activity.martial-arts-uniform,2016-03-28 16:36:43.269000+00:00,2024-06-15 19:13:43.041000+00:00,https://lichess.org/@/DrVitman,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,unicorn7love,Unicorn7Love,IM,nature.zebra,2020-08-19 09:57:26.628000+00:00,2024-06-16 19:19:48.241000+00:00,https://lichess.org/@/Unicorn7Love,True,False,False,...,Веду ютуб-канал. Тренер :) https://www.donatio...,Чеклецов,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,drawdenied_twitch,DrawDenied_Twitch,GM,activity.sparkles,2020-07-31 13:04:03.304000+00:00,2024-06-16 21:45:44.066000+00:00,https://lichess.org/@/DrawDenied_Twitch,True,False,False,...,https://www.twitch.tv/drawdenied\r\nhttps://im...,Krämer,True,https://www.twitch.tv/drawdenied,NaN,NaN,NaN,NaN,NaN,NaN
3,rakhmanov_aleksandr,Rakhmanov_Aleksandr,GM,NaN,2020-03-22 14:28:05.834000+00:00,2024-06-17 08:20:49.790000+00:00,https://lichess.org/@/Rakhmanov_Aleksandr,True,False,False,...,NaN,Rakhmanov,True,https://www.twitch.tv/gm_rakhmanov,4.0,1432.0,301.0,0.0,True,NaN
4,kurald_galain,Kurald_Galain,NaN,NaN,2023-05-19 16:12:20.116000+00:00,2024-06-17 08:25:02.015000+00:00,https://lichess.org/@/Kurald_Galain,True,False,False,...,Warren of Darkness and Elder Night,Rake,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# top 10 classic detail data
top10_classic_df = pd.DataFrame()
for i in top10_classic["id"]:
    top10_classic_df = pd.concat([top10_classic_df, pd.json_normalize(client.users.get_public_data(i))], ignore_index=True)

top10_classic_df.head()

,id,username,title,flair,patron,createdAt,seenAt,url,followable,following,...,perfs.racingKings.games,perfs.racingKings.rating,perfs.racingKings.rd,perfs.racingKings.prog,perfs.racingKings.prov,profile.uscfRating,playing,perfs.bullet.prov,perfs.blitz.prov,perfs.rapid.prov
0,unicorn7love,Unicorn7Love,IM,nature.zebra,True,2020-08-19 09:57:26.628000+00:00,2024-06-16 19:19:48.241000+00:00,https://lichess.org/@/Unicorn7Love,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,kanake04,Kanake04,NaN,NaN,NaN,2023-03-02 18:07:49.505000+00:00,2024-06-16 21:05:41.533000+00:00,https://lichess.org/@/Kanake04,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,vlad_lazarev79,Vlad_Lazarev79,NaN,NaN,NaN,2023-08-11 03:09:41.891000+00:00,2024-06-16 17:57:57.546000+00:00,https://lichess.org/@/Vlad_Lazarev79,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ojaijoao,OjaiJoao,FM,food-drink.bell-pepper,NaN,2016-09-02 02:01:40.503000+00:00,2024-06-17 04:42:38.199000+00:00,https://lichess.org/@/OjaiJoao,True,False,...,20.0,1872.0,202.0,-3.0,True,2404.0,NaN,NaN,NaN,NaN
4,igormezentsev,igormezentsev,FM,NaN,NaN,2017-07-23 04:04:13.003000+00:00,2024-06-17 08:26:28.931000+00:00,https://lichess.org/@/igormezentsev,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://lichess.org/4A5DjYiT/white,NaN,NaN,NaN


In [ ]:
# top 10 ultra bullet detail data
top10_ultrabullet_df = pd.DataFrame()
for i in top10_ultrabullet["id"]:
    top10_ultrabullet_df = pd.concat([top10_ultrabullet_df, pd.json_normalize(client.users.get_public_data(i))], ignore_index=True)

top10_ultrabullet_df.head()

,id,username,title,flair,createdAt,seenAt,url,followable,following,blocking,...,perfs.horde.prov,perfs.racingKings.games,perfs.racingKings.rating,perfs.racingKings.rd,perfs.racingKings.prog,perfs.racingKings.prov,streamer.youTube.channel,playing,patron,perfs.bullet.prov
0,aaryan_varshney,aaryan_varshney,FM,activity.1st-place-medal,2019-04-21 17:09:04.962000+00:00,2024-06-17 07:39:01.059000+00:00,https://lichess.org/@/aaryan_varshney,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blazinq,Blazinq,FM,NaN,2017-09-01 20:56:20.157000+00:00,2024-06-17 08:54:49.398000+00:00,https://lichess.org/@/Blazinq,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ohanyaneminchess,OhanyanEminChess,GM,symbols.exclamation-question-mark,2019-08-20 15:29:47.375000+00:00,2024-06-17 06:35:55.484000+00:00,https://lichess.org/@/OhanyanEminChess,True,False,False,...,True,375.0,1981.0,152.0,13.0,True,https://www.youtube.com/channel/UCH6VP4qtJEMNg...,NaN,NaN,NaN
3,tamojerry,tamojerry,FM,NaN,2020-09-22 17:53:13.817000+00:00,2024-06-17 07:10:41.292000+00:00,https://lichess.org/@/tamojerry,True,False,False,...,NaN,19.0,1748.0,217.0,31.0,True,NaN,NaN,NaN,NaN
4,ediz_gurel,Ediz_Gurel,GM,smileys.alien,2023-03-15 16:29:46.697000+00:00,2024-06-14 18:53:40.215000+00:00,https://lichess.org/@/Ediz_Gurel,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Function to get latest 100 games

In [ ]:
start = berserk.utils.to_millis(datetime(2023, 1, 1))
end = berserk.utils.to_millis(datetime(2023, 12, 31))

top1classic_game = client.games.export_by_player(top10_classic["id"][0], since=start, until=end)
top1classic_game

<generator object Games.export_by_player at 0x000001418ED9B4A0>

In [ ]:
games = list(_)
games[0]["createdAt"]

datetime.datetime(2023, 12, 26, 14, 27, 16, 517000, tzinfo=datetime.timezone.utc)

In [ ]:
games[-1]["createdAt"]

datetime.datetime(2023, 1, 1, 14, 28, 36, 683000, tzinfo=datetime.timezone.utc)

In [ ]:
game_id = games[0]["id"]
client.games.export(game_id)

{'id': 'QQrFH08R',
 'rated': False,
 'variant': 'standard',
 'speed': 'rapid',
 'perf': 'rapid',
 'createdAt': datetime.datetime(2023, 12, 26, 14, 27, 16, 517000, tzinfo=datetime.timezone.utc),
 'lastMoveAt': datetime.datetime(2023, 12, 26, 14, 29, 51, 922000, tzinfo=datetime.timezone.utc),
 'status': 'mate',
 'source': 'friend',
 'players': {'white': {'user': {'name': 'Unicorn7Love',
    'title': 'IM',
    'flair': 'nature.zebra',
    'patron': True,
    'id': 'unicorn7love'},
   'rating': 2699,
   'provisional': True,
   'analysis': {'inaccuracy': 1, 'mistake': 0, 'blunder': 0, 'acpl': 62}},
  'black': {'user': {'name': 'TumanyanArtem', 'id': 'tumanyanartem'},
   'rating': 965,
   'provisional': True,
   'analysis': {'inaccuracy': 0, 'mistake': 1, 'blunder': 1, 'acpl': 170}}},
 'winner': 'white',
 'opening': {'eco': 'C20',
  'name': "King's Pawn Game: Wayward Queen Attack",
  'ply': 3},
 'moves': 'e4 e5 Qh5 g6 Qxe5+ Be7 Qxh8 Kf8 Bc4 Bf6 Qxh7 Qe7 Nc3 Bxc3 dxc3 c6 Bg5 Qxg5 Qxf7#',
 'cl

In [ ]:
game_id2 = games[1]["id"]
client.games.export(game_id2)

{'id': 'DXYYoh6P',
 'rated': False,
 'variant': 'standard',
 'speed': 'rapid',
 'perf': 'rapid',
 'createdAt': datetime.datetime(2023, 12, 26, 14, 21, 23, 393000, tzinfo=datetime.timezone.utc),
 'lastMoveAt': datetime.datetime(2023, 12, 26, 14, 27, 11, 948000, tzinfo=datetime.timezone.utc),
 'status': 'mate',
 'source': 'friend',
 'players': {'white': {'user': {'name': 'TumanyanArtem',
    'id': 'tumanyanartem'},
   'rating': 965,
   'provisional': True,
   'analysis': {'inaccuracy': 1, 'mistake': 2, 'blunder': 8, 'acpl': 211}},
  'black': {'user': {'name': 'Unicorn7Love',
    'title': 'IM',
    'flair': 'nature.zebra',
    'patron': True,
    'id': 'unicorn7love'},
   'rating': 2699,
   'provisional': True,
   'analysis': {'inaccuracy': 1, 'mistake': 2, 'blunder': 7, 'acpl': 171}}},
 'winner': 'black',
 'opening': {'eco': 'C50',
  'name': 'Four Knights Game: Italian Variation',
  'ply': 8},
 'moves': 'e4 e5 Nc3 Nc6 Nf3 Nf6 Bc4 Bc5 O-O d6 Ne2 Bg4 Nh4 Bxe2 Qxe2 g5 Nf5 Qd7 Qf3 g4 Qb3 Na5